In order to speed up processing time when running classification algorithms, it is often useful to choose only the most "best" genes to use.  There are various algorithms available to choose genes, however here we use Chi2 Select K best.  K is how many genes you wish to use for testing stability.  More genese is usually better, however again in order to speed up processing time we limit the number of genes used.  This program allows you to set a min and max number of genes and an interval.  This will in turn setup numpy arrays with class and the select number of genes for further processing by FASTR and FASTrand.

### Libraries
Must be pre-installed.  Recommended to use virtual environment.

In [4]:
import numpy as np
from enum import Enum
from random import shuffle, sample
from os import path, getcwd, makedirs
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.model_selection import StratifiedKFold
from collections import Counter
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import lsqr
from math import sqrt, floor
from random import uniform, choice
from sklearn import svm
from sklearn import datasets
import multiprocessing as mp

## Methods and Classes

In [5]:
class AlterStrategy(Enum):
    GREEDY = 0
    CHI2 = 1
    RAND = 2
    PERCENT = 3

### NBC.py

In [6]:
class Model:
    """Describes the model class."""

    def __init__(self, samples, eps, class_label):
        """Initialize the model class.

        Args:
            samples: training samples of size [samples, genes].
            eps: epsilon value for correlation cutoff.
            class_label: classification label.
        """

        self.class_label = class_label
        self.samples = np.array(samples)
        self.eps = eps

        # columns are variables, rows are samples
        self.correlation = np.corrcoef(self.samples, y=None, rowvar=False)

        # note that the mask is actually the graph
        self.mask = (np.absolute(self.correlation) > self.eps)

        # the coefficients associated with the system of equation: Ax=b,
        # where A is an equation list created from the neighbors of gene
        # n and b is the value of gene n.
        self.geneFuncMasks = []  # these are the coefficients in Ax=b
        for gene in range(len(self.correlation)):
            currMask = self.mask[gene]
            setOfNeighbors = []
            solutions = []
            for sample in self.samples:
                neighbors = [sample[neighbor] if (currMask[neighbor] and (gene != neighbor))
                             else 0 for neighbor in range(len(currMask))]
                neighbors.append(1)
                setOfNeighbors.append(neighbors)
                solutions.append(sample[gene])
            coeff = self.solver(setOfNeighbors, solutions, 2)
            self.geneFuncMasks.append(coeff.tolist())

        self.coefficients = np.array(self.geneFuncMasks)

    def solver(self, neighbors, sols, choice):
        # Use lsqr to solve Ax=b
        A = np.array(neighbors)
        b = np.array(sols)
        x = lsqr(A, b)[0]
        return x

    def expression(self, sample):
        """Given a sample, return the hypothetical expression.

        Args:
            sample: the sample whose hypothetical expression we wish to
            calculate
        Returns:
            expr: A list with the expression values of size number of genes.
        """
        expression = []
        for gene in range(len(self.coefficients)):
            geneVal = 0
            for neighbor in range(len(self.mask)-1):
                geneVal += self.coefficients[gene][neighbor] * sample[neighbor]
            geneVal += self.coefficients[gene][len(self.mask)]
            expression.append(geneVal)
        return np.array(expression)

    def label (self):
        """Return the classification label of this model."""
        return self.class_label


class NetworkBasedClassifier:
    """Describes the NBClassifier class."""

    def __init__(self, epsilon):
        """Initialize a NBF classifier.

        Args:
            eps: epsilon value
        """
        self.models = []
        self.epsilon = epsilon

    def fit(self, X, y):
        """Fit the data with classes to create class models.

        Fits the data [num_samples, num_genes] with classifications
        [num_samples] to the model.  Creates as many models as classes.

        Args:
            X: the data we wish to train the classifier on
            y: the classifications associated with the samples
        """
        y = np.array(y)
        X = np.array(X)
        for key in Counter(y):
            a_class = np.where(y == key)
            self.models.append(Model([X[i] for i in a_class[0]], self.epsilon, key))

            
    def score(self, X, y):
        """Scores the classifications of a given set of samples (X) according to their
        actual clsssifications (y).

        Must fit the classifier before this method is called.

        Args:
            samples: the samples we wish to predict classification for.

        Returns:
            accuracy: the classification accuracy
        """
        y = np.array(y)
        X = np.array(X)
        predicted = self.predict(X)        
        correct = np.asarray(predicted == y)
        return np.sum(correct)/correct.shape[0]        
        
        
    def predict(self, X):
        """Predict the classification of a sample.

        Must fit the classifier before this method is called.

        Args:
            samples: the samples we wish to predict classification for.

        Returns:
            classifications: the classifications of the samples.
        """
        classifications = []
        for sample in X:
            RMSEs = []
            for model in self.models:
                rmse = sqrt( mean_squared_error(sample, model.expression(sample)))
                RMSEs.append(rmse)
            min_index = RMSEs.index(min(RMSEs))
            label = self.models[min_index].label()
            classifications.append(label)
        return np.array(classifications)

### Alter.py

In [7]:
def Subset (X, percent):
    """
    B/c genese are already ordered by chi2 rank we can choose top k to alter.
    """
    result = []
    idx = floor(X[0].size * percent)
    if idx <= 0:
        return X
    else:
        for x in X:
            alt = np.copy(x)
            for i in range(idx):
                alt[i] = choice([0, alt[i]*2])
            result.append(alt)
        return np.array(result)

In [11]:
def RandSubset (X, percent):
    result = []
    k = floor(X[0].size * percent)
    if k <= 0:
        return X
    else:
        indices = sample(range(X[0].size),k)
        for x in X:
            alt = np.copy(x)
            for i in indices:
                alt[i] = choice([0, alt[i]*2])
            result.append(alt)
        return np.array(result)

In [12]:
def All (X, percent):
    result = []
    for x in X:
        alt = []
        for gene in x:
            _offset = gene * percent
            _low = gene - _offset
            _high = gene + _offset
            alt.append(choice([_low, _high]))
        result.append(alt)
    return np.array(result)

In [24]:
def SelectKBestRanked(k, X, y):    
    b = SelectKBest(chi2, k).fit(X, y)
    a = b.get_support(indices = True)
    a = [x for _,x in sorted(zip(b.scores_[a],a),reverse=True)]
    return np.array(a)

In [8]:
def Accuracy(fit_estimator, X, y, chosen, idx_to_change):
    # TODO: run multiple times and return avg accuracy
    result = []
    for x in X:
        alt = np.copy(x)
        # alter prev chosen
        for i in chosen:
            alt[i] = choice([0, alt[i]*2]) 
        # alter new gene
        alt[idx_to_change] = choice([0, alt[idx_to_change]*2]) 
        result.append(alt)
    result = np.array(result)
    return fit_estimator.score(result, y)

In [25]:
def KBestGreedy(k, fit_estimator, X, y):
    notChosen = list(range(X.shape[1]))
    chosen = []
    for i in range(k):
        pool = mp.Pool(processes = mp.cpu_count())
        accuracy = pool.starmap(Accuracy, [(fit_estimator, X, y, chosen, idx) for idx in notChosen])
        a = [x for _,x in sorted(zip(accuracy, notChosen))]
        chosen.append(a[0])
        notChosen.remove(a[0])  
    return chosen

In [10]:
def Greedy (percent, fit_estimator, X, y):
    result = []
    k = floor(X[0].size * percent)
    if k <= 0:
        return X
    else:
        indices = KBestGreedy(k, fit_estimator, X, y)
        for x in X:
            alt = np.copy(x)
            for i in indices:
                alt[i] = choice([0, alt[i]*2])
            result.append(alt)
        return np.array(result)

## START MAIN PROGRAM

###### Enter the series and feature_size to use
Must be all upper case. e.g. `"GSE27562"`

In [14]:
series = "GSE19804"
feature_size = 50
fs_strategy = fsStrategy.KBEST

NameError: name 'fsStrategy' is not defined

### Get/Create Directories
Assumes this notebook is in `GenClass-Stability/main/notebooks/`

In [15]:
notebook_dir = getcwd();
main_dir = path.dirname(path.dirname(notebook_dir))
load_path = path.join(main_dir, "GSE", series)
gsa_path = path.join(main_dir,"GSA", series, str(feature_size))
if not path.exists(gsa_path):
    makedirs(gsa_path)

### Import Classes and Expressions
Load original data. Assumes SIT and custome GSE script have been run to import data.

In [16]:
classes =np.loadtxt(path.join(load_path, "classes.txt"), dtype=np.str, delimiter="\t")
exprs = np.loadtxt(path.join(load_path, "exprs.txt"), delimiter="\t")

Select K best genes for analysis.

In [17]:
a = SelectKBestRanked(feature_size, exprs, classes)

In [18]:
exprs = exprs[:, a]
exprs.shape

(120, 50)

Save the selected expression data for potential later use.

In [19]:
np.save(path.join(gsa_path,"exprs.npy"), exprs)
np.save(path.join(gsa_path,"classses.npy"), classes)

## Stability Test I and II and III

In [21]:
def crossValidate(estimator, X, y, cv=10, alt=0, alterStrat = Greedy):
    scores = []
    skf = StratifiedKFold(cv)
    for train_index, test_index in skf.split(X, y):
        estimator.fit(X[train_index], y[train_index]) 
        accuracy = estimator.score(
            alterStrat(alt, estimator, X[test_index], y[test_index]),
            y[test_index])
        scores.append(accuracy)
    return np.array(scores)

In [26]:
svc = NetworkBasedClassifier( 0.8 )
scores = crossValidate(svc, exprs, classes, 10, 0.1, Greedy)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9166666666666666, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9166666666666666, 0.9166666666666666, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.9166666666666666, 1.0, 1.0, 0.9166666666666666, 1.0, 0.5, 1.0, 0.9166666666666666, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[41, 7, 18, 19, 36, 39, 43, 0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 37, 38, 40, 42, 44, 45, 46, 47, 48, 49]
[41]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49]
[0.5833333333333334, 0.5, 0.5833333333333334, 0.5833333333333334, 0.5, 0.583333333

[41, 1, 4, 2]
[0, 3, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49]
[0.5833333333333334, 0.6666666666666666, 0.5, 0.5, 0.75, 0.5833333333333334, 0.5833333333333334, 0.5, 0.5, 0.8333333333333334, 0.5833333333333334, 0.5, 0.5, 0.6666666666666666, 0.6666666666666666, 0.75, 0.5833333333333334, 0.5833333333333334, 0.5833333333333334, 0.5, 0.6666666666666666, 0.75, 0.5833333333333334, 0.6666666666666666, 0.5833333333333334, 0.5833333333333334, 0.6666666666666666, 0.5833333333333334, 0.5833333333333334, 0.6666666666666666, 0.75, 0.5833333333333334, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5833333333333334, 0.5, 0.5833333333333334, 0.6666666666666666, 0.5, 0.75, 0.5, 0.5833333333333334, 0.5]
[5, 6, 10, 11, 14, 15, 22, 35, 36, 37, 38, 39, 42, 45, 47, 49, 0, 8, 9, 13, 19, 20, 21, 25, 27, 28, 30, 31, 34, 40, 43, 48, 3, 16, 17, 23, 26, 29, 32, 44, 7, 18, 24, 33, 46, 12]
[]
[0, 1, 2, 3, 4, 

[41, 7]
[0, 1, 2, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49]
[0.6666666666666666, 0.5, 0.5, 0.6666666666666666, 0.5833333333333334, 0.5, 0.5833333333333334, 0.6666666666666666, 0.5833333333333334, 0.6666666666666666, 0.75, 0.6666666666666666, 0.5, 0.5833333333333334, 0.5833333333333334, 0.6666666666666666, 0.75, 0.5833333333333334, 0.5833333333333334, 0.75, 0.6666666666666666, 0.5833333333333334, 0.5, 0.5833333333333334, 0.5833333333333334, 0.5833333333333334, 0.5833333333333334, 0.6666666666666666, 0.5, 0.6666666666666666, 0.5833333333333334, 0.6666666666666666, 0.5, 0.5833333333333334, 0.6666666666666666, 0.5833333333333334, 0.5833333333333334, 0.5, 0.5833333333333334, 0.5, 0.6666666666666666, 0.5833333333333334, 0.6666666666666666, 0.5833333333333334, 0.5833333333333334, 0.75, 0.6666666666666666, 0.5833333333333334]
[1, 2, 5, 13, 23, 29, 33, 38, 40, 4, 6

[]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49]
[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 0.75, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0]
[41, 0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49]
[41]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49]
[0.6666666666666666, 0.5833333333333334, 0.5833333333333334, 0.5833333333333334, 0.5, 0.5833333333333334, 0.6666666666666666, 0.5833333333333334, 0.6666666666666666, 0.583

[41, 7, 2]
[0, 1, 3, 4, 5, 6, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49]
[0.5833333333333334, 0.6666666666666666, 0.6666666666666666, 0.5, 0.6666666666666666, 0.5, 0.5833333333333334, 0.5, 0.5833333333333334, 0.5833333333333334, 0.6666666666666666, 0.6666666666666666, 0.75, 0.5833333333333334, 0.5833333333333334, 0.5, 0.5, 0.4166666666666667, 0.6666666666666666, 0.6666666666666666, 0.5833333333333334, 0.6666666666666666, 0.5833333333333334, 0.5833333333333334, 0.6666666666666666, 0.5, 0.75, 0.6666666666666666, 0.5833333333333334, 0.6666666666666666, 0.5833333333333334, 0.5, 0.5833333333333334, 0.5833333333333334, 0.6666666666666666, 0.5833333333333334, 0.5, 0.5833333333333334, 0.6666666666666666, 0.5833333333333334, 0.5833333333333334, 0.6666666666666666, 0.5833333333333334, 0.5833333333333334, 0.6666666666666666, 0.5, 0.5833333333333334]
[19, 4, 6, 9, 17, 18, 27, 33, 

[41]
[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49]
[0.5833333333333334, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5833333333333334, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5833333333333334, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5, 0.5]
[1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49, 0, 8, 17]
[41, 1]
[0, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 42, 43, 44, 45, 46, 47, 48, 49]
[0.5833333333333334, 0.5, 0.5, 0.5, 0.5833333333333334, 0.6666666666666666, 0.5, 0.5833333333333334, 0.5833333333333334, 0.583333333333333

[41, 10, 2, 39]
[0, 1, 3, 4, 5, 6, 7, 8, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 40, 42, 43, 44, 45, 46, 47, 48, 49]
[0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.75, 0.6666666666666666, 0.5, 0.5833333333333334, 0.6666666666666666, 0.5833333333333334, 0.5, 0.5833333333333334, 0.5833333333333334, 0.75, 0.5833333333333334, 0.5, 0.8333333333333334, 0.5833333333333334, 0.5833333333333334, 0.5833333333333334, 0.5833333333333334, 0.75, 0.5833333333333334, 0.5, 0.5833333333333334, 0.5, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.5, 0.6666666666666666, 0.6666666666666666, 0.6666666666666666, 0.5, 0.5833333333333334, 0.5833333333333334, 0.6666666666666666, 0.6666666666666666, 0.5, 0.5833333333333334, 0.6666666666666666, 0.5833333333333334, 0.5833333333333334, 0.6666666666666666, 0.5833333333333334, 0.6666666666666666, 0.6666666666666666]
[6, 11, 16, 24, 26, 30, 34, 40, 7, 9, 12, 13, 15,